##Основаная часть кода:

In [ ]:
# Импорт необходимых библиотек.
import numpy as np
import tensorflow as tf
import pandas as pd
from nltk.tokenize import sent_tokenize
import os
import re

In [ ]:
# Установка библиотеки для обработки текстов.
!pip install deeppavlov

In [ ]:
# Скачивание надстройки.
!python -m deeppavlov install morpho_ru_syntagrus_pymorphy

In [ ]:
# Создание папки с результатами.
os.mkdir("Results")
os.chdir("Results")
os.mkdir("Metadata")
os.mkdir("Morphologized words")
os.mkdir("Tokenized sentences")
os.chdir("..")

In [ ]:
# Создание папки для хранения текстовых данных.
os.mkdir("Text_Data")
os.chdir("Text_Data")

In [ ]:
# Основная часть алгоритма.

# Создаем list со всеми именами файлов в папке "Text_Data"
LoF = os.listdir()

# Создаем модель для лемматизации слов.
from deeppavlov import build_model, configs
model_tr = build_model(configs.morpho_tagger.UD2_0.morpho_ru_syntagrus_pymorphy_lemmatize, download=True)

# Создаем DataFrame, в котором будут храниться meta-данные.
df_meta = pd.DataFrame(index=range(len(LoF)), columns=["Text id", "Number of tokens", "Number of lemmas", "Number of sentences", "TTR", "Discipline"])

# Начинаем обработку файлов в цикле.
for FileIndex in range(len(LoF)):

  # Создаем необходимые переменные.
  dict_tkn = dict()
  dict_lem = dict()
  tknz_sent = list()
  st_tr = str()

  # Выгружаем текстовый файл в list.
  fin = open(LoF[FileIndex], 'r', encoding='utf8')
  txt = fin.readlines()
  fin.close()

  # Предобрабатываем текст.
  for i in range(len(txt)):
    txt[i] = txt[i].replace("\ufeff", "")
    txt[i] = txt[i].replace("\n", "")
    txt[i] = txt[i].replace("\t", "")

  # Создем список предложений.
  tknz_sent = ''.join(txt)
  tknz_sent = sent_tokenize(tknz_sent)

  # Обрабатываем текстовые данные и сохраняем результат в строку.
  for parse in model_tr(txt):
    st_tr += parse

  # Начинаем обработку полученной строки, форматируя данные и очищая спец. символы.
  lst_tr = st_tr.split('\n')
  rs_tr = list()

  for part in lst_tr:
    part_fragmented = part.split('\t')

    # Проверка нужна только для конца, потом что-нибудь придумаю.
    if len(part_fragmented) >= 2:
      rs_tr += [part_fragmented[0:4] + part_fragmented[5:6]]
  
  # Выгружаем слова и леммы в словарь.
  for i in range(len(rs_tr)):

    # Загрузка слова.
    if rs_tr[i][1] in dict_tkn:
      dict_tkn[rs_tr[i][1]] += 1
    else:
      dict_tkn[rs_tr[i][1]] = 1

    # Загрузка леммы.
    if rs_tr[i][2] in dict_lem:
      dict_lem[rs_tr[i][2]] += 1
    else:
      dict_lem[rs_tr[i][2]] = 1

  # Добавляем строку данных в meta-данные.
  TTR = len(dict_lem) / len(dict_tkn)
  df_meta.iloc[FileIndex] = [FileIndex + 1, len(dict_tkn), len(dict_lem), len(tknz_sent), TTR, "Sociologia"]

  # Начинаем выгружать результаты.

  # В папке "Morphologized words" находятся xlsx-файл с разобранными словами из текстового файла.
  df_mrph = pd.DataFrame(rs_tr, columns=["Index", "Word", "Lemma", "Part of speech", "Facts"])

  os.chdir("../Results/Morphologized words")
  df_mrph.to_excel(LoF[FileIndex][0 : len(LoF[FileIndex]) - 4] + "_morph" + ".xlsx")
  os.chdir("..")
  os.chdir("../Text_Data")
  
  # Папка "Tokenized sentences" cодержит в себе текст, который был разбит на предложения.
  df_snt = pd.DataFrame(tknz_sent, columns=["Sentences"])

  os.chdir("../Results/Tokenized sentences")
  df_snt.to_excel(LoF[FileIndex][0 : len(LoF[FileIndex]) - 4] + "_sentences" + ".xlsx")
  os.chdir("..")
  os.chdir("../Text_Data")

# В папке "Metadata" будут количественные данные об анализируемом тексте.
# Выгружаем результаты из df_meta после работы алгоритма.
os.chdir("../Results/Metadata")
df_meta.to_excel("Sociologia" + "_metadata" + ".xlsx")
os.chdir("..")
os.chdir("../Text_Data")

In [ ]:
# Скачивание результатов.
os.chdir("..")
!zip -r ./Results.zip '/content/Results'

from google.colab import files
files.download("/content/Results.zip")

os.chdir("Text_Data")